In [1]:
import os

In [2]:
BUCKET = 'gs://edml/data/taxi-trips'

In [3]:
os.environ['BUCKET'] = BUCKET

## Local training / Python

In [4]:
%%bash
export PYTHONPATH=${PYTHONPATH}:~/event-driven-ml/edml-trainer
echo $PYTHONPATH

:/home/jupyter/event-driven-ml/edml-trainer


In [7]:
%%bash
echo "bucket=${BUCKET}"
rm -rf model_trained_test
export PYTHONPATH=${PYTHONPATH}:~/event-driven-ml/edml-trainer
python3 -m trainer.task \
  --bucket=${BUCKET} \
  --output-dir=model_trained_test \
  --job-dir=./tmp \
  --pattern="*" \
  --train-examples=174000 \
  --nembeds 10 \
  --nnsize 10 5 \
  --eval-steps=2

bucket=gs://edml/data/taxi-trips
Will train for 39.0625 steps using batch_size=128
Will use DNN size of [10, 5]


INFO:tensorflow:Using config: {'_service': None, '_train_distribute': None, '_save_summary_steps': 100, '_model_dir': 'model_trained_test', '_master': '', '_keep_checkpoint_every_n_hours': 10000, '_num_ps_replicas': 0, '_evaluation_master': '', '_protocol': None, '_global_id_in_cluster': 0, '_session_creation_timeout_secs': 7200, '_tf_random_seed': 2810, '_save_checkpoints_secs': 300, '_log_step_count_steps': 100, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 3, '_experimental_distribute': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fcc6d7c9c18>, '_is_chief': True, '_num_worker_replicas': 1, '_task_id': 0, '_task_type': 'worker', '_device_fn': None, '_eval_distribute': None, '_experimental_max_worker_delay_secs': None}
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, 

In [8]:
from google.datalab.ml import TensorBoard

In [9]:
TensorBoard().start('./model_trained_test')
TensorBoard().list()

TensorBoard was started successfully with pid 4515. Click here to access it.

,logdir,pid,port
0,./model_trained_test,4515,33815


In [10]:
for pid in TensorBoard.list()['pid']:
    TensorBoard().stop(pid)
    print('Stopped TensorBoard with pid {}'.format(pid))

Stopped TensorBoard with pid 4515


In [11]:
%%bash
export MODEL_DIR=tmp_tensorboard
tensorboard --logdir=$MODEL_DIR --port=6006

Process is terminated.


## Local prediction

In [12]:
%%writefile inputs.json
{"uuid": "b1", "dayofweek": 6, "hourofday": 7, "pickup_zone_name": "World Trade Center", "dropoff_zone_name": "Newark Airport", "passenger_count": 1}
{"uuid": "g1", "dayofweek": 3, "hourofday": 23, "pickup_zone_name": "World Trade Center", "dropoff_zone_name": "Times Sq/Theatre District", "passenger_count": 1}

Writing inputs.json


In [13]:
%%bash
MODEL_LOCATION=$(ls -d $(pwd)/model_trained_test/export/exporter/* | tail -1)
echo $MODEL_LOCATION
gcloud ai-platform local predict --model-dir=$MODEL_LOCATION --json-instances=inputs.json

/home/jupyter/event-driven-ml/edml-notebooks/gbi/model_trained_test/export/exporter/1573144860
PREDICTIONS
[7.751214981079102]
[8.675673484802246]


If the signature defined in the model is not serving_default then you must specify it via --signature-name flag, otherwise the command may fail.






2019-11-07 16:41:46.827335: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  AVX2 FMA
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2019-11-07 16:41:46.836000: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-11-07 16:41:46.836275: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55ed8b41f9f0 executing computations on platform Host. Devices:
2019-11-07 16:41:46.836305: I tensorflow/compiler/xla/service/service.cc:175]   StreamExecutor device (0): <undefined>, <undefined>
2019-11-07 16:41:46.836613: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter 

## Local training / Gcloud

In [14]:
%%bash
rm -rf model_trained_test
gcloud ai-platform local train \
   --module-name=trainer.task \
   --package-path=~/event-driven-ml/edml-trainer \
   --job-dir=./tmp \
   -- \
   --train-examples=500 \
   --output-dir=model_trained_test \
   --pattern="*" \
   --eval-steps=1

ERROR: (gcloud.ai-platform.local.train) /usr/bin/python: command not found


CalledProcessError: Command 'b'rm -rf model_trained_test\ngcloud ai-platform local train \\\n   --module-name=trainer.task \\\n   --package-path=~/event-driven-ml/edml-trainer \\\n   --job-dir=./tmp \\\n   -- \\\n   --train-examples=500 \\\n   --output-dir=model_trained_test \\\n   --pattern="*" \\\n   --eval-steps=1\n'' returned non-zero exit status 1